In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, SolverFactory

# Create a ConcreteModel
model = ConcreteModel()

# Parameters
C_internal = 750000  # Internal maintenance cost per fault type
C_external_normal = 50000  # External maintenance cost in normal season
C_external_high_demand = 150000  # External maintenance cost in high demand season
p_internal = 0.6  # Preventive success rate
D_failure = 200000  # Downtime cost per failure (example value, you can adjust)
F_freq = 100  # Frequency of preventable faults (example value)
U_required = 1000  # Required uptime (example value)
T_total = 2000  # Total available time (example value)
B = 5000000  # Budget (example value)

# Decision variables
model.x_internal = Var(domain=NonNegativeReals)
model.x_external_normal = Var(domain=NonNegativeReals)
model.x_external_high_demand = Var(domain=NonNegativeReals)
model.F_occur = Var(domain=NonNegativeReals)  # Number of failures due to unprevented issues

# Objective function: Minimize total cost
model.TotalCost = Objective(
    expr=(
        C_internal * model.x_internal +
        C_external_normal * model.x_external_normal +
        C_external_high_demand * model.x_external_high_demand +
        D_failure * model.F_occur
    ),
    sense=1  # Minimize
)

# Failure prevention model: F_occur = F_freq * (1 - p_internal * x_internal)
model.FailurePrevention = Constraint(
    expr=model.F_occur == F_freq * (1 - p_internal * model.x_internal)
)

# Uptime constraint: T_total - T_down >= U_required
model.UptimeConstraint = Constraint(
    expr=T_total - (D_failure * model.F_occur) >= U_required
)

# Budget constraint: Total maintenance cost <= B
model.BudgetConstraint = Constraint(
    expr=(
        C_internal * model.x_internal +
        C_external_normal * model.x_external_normal +
        C_external_high_demand * model.x_external_high_demand
    ) <= B
)

# Solve the model using Gurobi
solver = SolverFactory('glpk')  # Gurobi solver
result = solver.solve(model, tee=True)

# Display results
print("Optimal Solution:")
print(f"Internal Maintenance Actions (x_internal): {model.x_internal.value}")
print(f"External Maintenance Actions (Normal) (x_external_normal): {model.x_external_normal.value}")
print(f"External Maintenance Actions (High Demand) (x_external_high_demand): {model.x_external_high_demand.value}")
print(f"Number of Failures (F_occur): {model.F_occur.value}")
print(f"Total Cost: {model.TotalCost.expr()}")
